In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding   
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf
import logging


tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 32
BATCH_SIZE = 256
INPUT_FILE_NAME = "../data/frankenstein.txt"
WINDOW_LENGHT = 40
WINDOW_STEP = 3
PREDICT_LENGTH = 3
MAX_WORDS = 10000
EMBEDDING_WIDTH = 100

In [38]:
file = open(INPUT_FILE_NAME, "r", encoding="utf-8-sig")
text = file.read()
file.close()

text = text_to_word_sequence(text)

fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGHT, WINDOW_STEP):
    fragments.append(text[i : i + WINDOW_LENGHT])
    targets.append(text[i + WINDOW_LENGHT])

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="UNK")
tokenizer.fit_on_texts(text)
fragments_indexed = tokenizer.texts_to_sequences(fragments)
targets_indexed = tokenizer.texts_to_sequences(targets)

X = np.array(fragments_indexed, dtype=np.int64)
y = np.zeros((len(targets_indexed), MAX_WORDS))
for i, target_index in enumerate(targets_indexed):
    y[i, target_index] = 1



In [39]:
training_model = Sequential()
training_model.add(Embedding(output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS, mask_zero=True, input_length=None))
training_model.add(LSTM(128,return_sequences=True, dropout= 0.2, recurrent_dropout=0.2))
training_model.add(LSTM(128, dropout= 0.2, recurrent_dropout=0.2))
training_model.add(Dense(128,activation='relu'))
training_model.add(Dense(MAX_WORDS,activation='softmax'))
training_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
training_model.summary()
history = training_model.fit(X,y, validation_split = 0.05, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 2, shuffle = True)

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/32
97/97 - 23s - 240ms/step - loss: 7.1898 - val_loss: 7.9966
Epoch 2/32
97/97 - 17s - 177ms/step - loss: 6.4024 - val_loss: 8.2081
Epoch 3/32
97/97 - 44s - 453ms/step - loss: 6.2744 - val_loss: 8.4663
Epoch 4/32
97/97 - 41s - 428ms/step - loss: 6.1505 - val_loss: 8.6937
Epoch 5/32
97/97 - 39s - 404ms/step - loss: 6.0207 - val_loss: 8.7270
Epoch 6/32
97/97 - 35s - 366ms/step - loss: 5.9192 - val_loss: 8.9046
Epoch 7/32
97/97 - 39s - 403ms/step - loss: 5.8446 - val_loss: 8.9316
Epoch 8/32
97/97 - 38s - 391ms/step - loss: 5.7733 - val_loss: 8.9699
Epoch 9/32
97/97 - 35s - 366ms/step - loss: 5.7010 - val_loss: 9.0660
Epoch 10/32
97/97 - 35s - 357ms/step - loss: 5.6160 - val_loss: 9.0327
Epoch 11/32
97/97 - 35s - 360ms/step - loss: 5.5287 - val_loss: 9.1245
Epoch 12/32
97/97 - 35s - 362ms/step - loss: 5.4448 - val_loss: 9.2134
Epoch 13/32
97/97 - 35s - 361ms/step - loss: 5.3736 - val_loss: 9.2280
Epoch 14/32
97/97 - 37s - 380ms/step - loss: 5.3080 - val_loss: 9.4068
Epoch 15/32
97/

In [40]:
from tensorflow.keras.layers import Input

In [41]:
inference_model = Sequential()
inference_model.add(Input(batch_shape=(1, 1)))
inference_model.add(Embedding(output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS, mask_zero=True))
inference_model.add(LSTM(128,return_sequences=True, dropout= 0.2, recurrent_dropout=0.2, stateful=True))
inference_model.add(LSTM(128, dropout= 0.2, recurrent_dropout=0.2))
inference_model.add(Dense(128,activation='relu'))
inference_model.add(Dense(MAX_WORDS,activation='softmax'))
weights = training_model.get_weights()
inference_model.set_weights(weights)


In [42]:
first_words = ['i', 'saw']

first_words_indexed = tokenizer.texts_to_sequences(first_words)
for layer in inference_model.layers:
    if hasattr(layer, 'reset_states'):
        layer.reset_states()
predicted_string = ''
for i ,word_index in enumerate(first_words_indexed):
    x = np.zeros((1,1), dtype=np.int64)
    x[0][0] = word_index[0]
    predicted_string+= first_words[i]
    predicted_string += ' '
    y_predict = inference_model.predict(x,verbose=0)[0]
for i in range(PREDICT_LENGTH):
    new_word_index = np.argmax(y_predict)
    word = tokenizer.sequences_to_texts([[new_word_index]])
    x[0][0] = new_word_index
    predicted_string += word[0]
    predicted_string += ' '
    y_predict = inference_model.predict(x,verbose=0)[0]
print(predicted_string)

i saw the name and 
